In [12]:
import os, requests

qurl = 'https://dapi.kakao.com/v2/local/search/address.json'
apikey = os.environ['KakaoAK']
params = {'query' : '판교역로 235'}
headers = {'Authorization' : f'KakaoAK {apikey}'}

req = requests
res = req.get(qurl, params=params, headers=headers)

params['query'] = '경북 안동시 풍천면 도청대로 455'
res = req.get(qurl, params=params, headers=headers)

#res.json()['documents'][0]['address']['x']

In [1]:
'''
START FROM HERE
test code up until here
'''
ad2ge_cache = {}

LINK_WUNIT = 60 # km
EXCEL2READ = 'QKD 투자비 산정.xlsx' # link data

In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel(EXCEL2READ, header=3)
df.drop(df.columns[-1], axis=1, inplace=True)
df.columns = ['type', 'u', 'v', 'uaddr', 'vaddr', 'distance']

df.dropna()['type']

0     MAIN Ring
24      MESH 추가
38      일반 Ring
Name: type, dtype: object

In [3]:
df.loc[0:23, 'type'] = ['M'] * len(df[0:24])
df.loc[24:37, 'type'] = ['X'] * len(df[24:38])
df.loc[38:, 'type'] = ['N'] * len(df[38:])

df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)

In [4]:
wlist = df['distance'].map(lambda d: (int(d)+(LINK_WUNIT-1))//LINK_WUNIT).tolist()
df['weight'] = wlist

In [18]:
# fix wrong address
df.replace('경기 광주시 행정타운로 50 광주시청 => 광주 서구 내방로 111', '광주 서구 내방로 111', inplace=True)
df.replace('경기 이천시 창천동 466-5', '경기 이천시 창전동 466-5', inplace=True)
df.replace('전북 무주구 적상면 사천리 525-6', '전북 무주군 적상면 사천리 525-6', inplace=True)

In [19]:
df

,type,u,v,uaddr,vaddr,distance,weight
0,M,서울(★서울청사),남양주(T중심국),서울 종로구 세종대로 209,경기 남양주시 금곡동 172-10,42.0,1
1,M,남양주(T중심국),S가평집중국,경기 남양주시 금곡동 172-10,경기 가평군 가평읍 읍내리 470-1,49.0,1
2,M,S가평집중국,춘천(강원도청),경기 가평군 가평읍 읍내리 470-1,강원 춘천시 중앙로 1,38.0,1
3,M,춘천(강원도청),S홍천중심국,강원 춘천시 중앙로 1,강원 홍천군 홍천읍 연봉리 476-3,47.0,1
4,M,S홍천중심국,원주(원주시청),강원 홍천군 홍천읍 연봉리 476-3,강원 원주시 시청로 1,57.0,1
5,M,원주(원주시청),단양(T중심국),강원 원주시 시청로 1,충북 단양군 단양읍 상진리 432,71.0,2
6,M,단양(T중심국),안동(경북도청),충북 단양군 단양읍 상진리 432,경북 안동시 풍천면 도청대로 455,84.0,2
7,M,안동(경북도청),구미(구미시청),경북 안동시 풍천면 도청대로 455,경북 구미시 송정대로 55,107.0,2
8,M,구미(구미시청),대구(대구시청),경북 구미시 송정대로 55,대구 중구 공평로 88,75.0,2
9,M,대구(대구시청),밀양교동통합국,대구 중구 공평로 88,경남 밀양시 교동 625-16,77.0,2


In [20]:
dfu = df[['u', 'uaddr']]
dfu.columns = ['n', 'addr']
dfv = df[['v', 'vaddr']]
dfv.columns = ['n', 'addr']
ad2ge = pd.concat([dfu, dfv], ignore_index=True).drop_duplicates().reset_index(drop=True)

ad2ge['lat'], ad2ge['lon'] = np.nan, np.nan

In [21]:
import os, requests

def get_geoloc(a):
    global ad2ge_cache
    if a in ad2ge_cache:
        latlon = ad2ge_cache[a]
        return (latlon['y'], latlon['x'])
    
    qurl = 'https://dapi.kakao.com/v2/local/search/address.json'
    apikey = os.environ['KakaoAK']
    params = {'query' : a}
    headers = {'Authorization' : f'KakaoAK {apikey}'}
    rjson = requests.get(qurl, params=params, headers=headers).json()['documents']
    if len(rjson) > 0:
        latlon = rjson[0]['address']
        ad2ge_cache[a] = latlon
        return (latlon['y'], latlon['x'])
    return (np.nan, np.nan)

def set_geoloc(dfgeo):
    dfgeo.loc[:,['lat','lon']] = ad2ge['addr'].map(lambda a: get_geoloc(a)).tolist()
    return dfgeo

In [26]:
set_geoloc(ad2ge)

,name,addr,lat,lon
0,서울(★서울청사),서울 종로구 세종대로 209,37.5749078897065,126.975207341063
1,남양주(T중심국),경기 남양주시 금곡동 172-10,37.637996392796,127.215611811154
2,S가평집중국,경기 가평군 가평읍 읍내리 470-1,37.8306780534692,127.513030149359
3,춘천(강원도청),강원 춘천시 중앙로 1,37.8800729197963,127.727907820318
4,S홍천중심국,강원 홍천군 홍천읍 연봉리 476-3,37.6818006615099,127.877321496969
5,원주(원주시청),강원 원주시 시청로 1,37.3419249898787,127.919722776646
6,단양(T중심국),충북 단양군 단양읍 상진리 432,36.9815610954036,128.353095580421
7,안동(경북도청),경북 안동시 풍천면 도청대로 455,36.576153048556,128.505368170958
8,구미(구미시청),경북 구미시 송정대로 55,36.1200152839962,128.34382132634
9,대구(대구시청),대구 중구 공평로 88,35.8715530700224,128.601220069308


In [23]:
ad2ge[ad2ge.lat.isna()]

,n,addr,lat,lon


In [28]:
ad2ge.rename(columns={'n' : 'name'}, inplace=True)
ad2ge.head(1)

,name,addr,lat,lon
0,서울(★서울청사),서울 종로구 세종대로 209,37.5749078897065,126.975207341063


In [25]:
net_addr = df
net_geo = ad2ge

with pd.ExcelWriter('net.xlsx', engine='xlsxwriter') as writer:
    net_addr.to_excel(writer, index=False, sheet_name='addr')
    net_geo.to_excel(writer, index=False, sheet_name='geo')
    writer.save()